In [1]:
# For making the example's imports work in this folder.
#import sys, os
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [2]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from examples.simple_mnist.model import MLP
from examples.simple_mnist.aggregate import Classifier, ClassifierSettings
from examples.simple_mnist.repository import Classifiers
from examples.simple_mnist.dataset import Digits
from examples.simple_mnist.handlers import subscriber

In [3]:
model = MLP(input_size=28*28, hidden_size=128, output_size=10, p=0.5)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
dataloaders = [
    ('train', DataLoader(Digits(train=True, normalize=True), batch_size=32, shuffle=True)),
    ('test', DataLoader(Digits(train=False, normalize=True), batch_size=32, shuffle=False))
]

In [4]:
from torchsystem import Callbacks
from torchsystem.metrics.average import Accuracy, Loss

settings = ClassifierSettings()
classifier = Classifier(model, criterion, optimizer, settings)
classifier.bind(subscriber)
repository = Classifiers()
callbacks = Callbacks(Loss(), Accuracy())

In [5]:
for epoch in range(2):
    for phase, loader in dataloaders:
        classifier.phase = phase
        classifier.fit(loader, callbacks) if phase == 'train' else classifier.evaluate(loader, callbacks)
        for callback in callbacks:
            classifier.publish({'epoch': epoch, 'phase': phase, 'metric': callback.name, 'average': callback.average.value}, topic='results')
            callback.reset()
repository.store(classifier)
classifier.publish('End of training-----------------', topic='end')

Batch: 300, Metric: loss, Value: 0.9124188423156738, Phase: train
Batch: 300, Metric: accuracy, Value: 0.7006250000000006, Phase: train
Batch: 600, Metric: loss, Value: 0.7068272829055786, Phase: train
Batch: 600, Metric: accuracy, Value: 0.7719270833333334, Phase: train
Batch: 900, Metric: loss, Value: 0.6119914650917053, Phase: train
Batch: 900, Metric: accuracy, Value: 0.8058333333333332, Phase: train
Batch: 1200, Metric: loss, Value: 0.5578536987304688, Phase: train
Batch: 1200, Metric: accuracy, Value: 0.8249739583333335, Phase: train
Batch: 1500, Metric: loss, Value: 0.5249641537666321, Phase: train
Batch: 1500, Metric: accuracy, Value: 0.8363958333333333, Phase: train
Batch: 1800, Metric: loss, Value: 0.4936557412147522, Phase: train
Batch: 1800, Metric: accuracy, Value: 0.8469791666666666, Phase: train
Results {'epoch': 0, 'phase': 'train', 'metric': 'loss', 'average': tensor(0.4869, grad_fn=<DivBackward0>)}
Results {'epoch': 0, 'phase': 'train', 'metric': 'accuracy', 'average'